# 목차
    1. [라이브러리 불러오기](#라이브러리-불러오기)
    2. [파일 불러오기 & API 키 받아오기](#파일--api-키-받아오기)
    3. [gpt 필드 전처리](#gpt)
    4. [gpt 필드 정리 & 저장](#생성-필드-합치고-저장하기)

# 라이브러리 불러오기

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openai
import json
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from pydantic import BaseModel
from typing import List, Optional

# 파일 & API 키 받아오기

In [3]:
current_directory = os.getcwd()
data_directory = os.path.join(current_directory, '..', 'preprocessing')
data_file_path = os.path.join(data_directory, '전처리_호텔_results_20241010_113155.pickle')
df = pd.read_pickle(data_file_path)

In [4]:
# 환경변수에서 'OPENAPI_KEY' 값을 받아옴
# openapi_key = os.getenv('OPEN_API_KEY')

# GPT

In [5]:
df['description_1'] = '제목 : ' + df['title'].astype(str) + ', 상품 게시 시간 : ' + df['post_time'].astype(str) + ', 상세 설명 : ' + df['description'].astype(str)

In [6]:
# Pydantic을 사용해 응답 데이터를 구조화할 모델을 정의합니다.
class Fields(BaseModel):
    expiration_stdate: Optional[str]
    expiration_endate: Optional[str]
    room_type: Optional[str]
    head_count: Optional[int]
    shipping_fee: Optional[int]
    transaction_method: Optional[str]
    market_price: Optional[int]
    options: Optional[str]
    parking: Optional[bool]
    check_in_time: Optional[str]
    check_out_time: Optional[str]
    stay_type: Optional[str]
    foreign_yn: Optional[str]
    event: Optional[str]
    advertise : Optional[bool]

# 함수를 정의하여 데이터를 요청하고 응답을 처리합니다.
def extract_fields(description):
    prompt = {
        "role": "system", 
        "content": """
        너는 데이터 필드를 추출해주는 도우미야. 데이터를 받아서 필요한 필드를 15개로 제한해 JSON 형식으로 응답할거야. 
        설명에서 필요한 15개의 필드를 정확히 추출하고 나머지 필드들은 무시해줘. 필드는 오직 15개만 있어야 하고, 추가적인 세부 필드를 포함해서는 안 돼.
        NaN이나 정보가 없으면 해당 필드를 null로 설정해줘.
        필드는 아래 15개:
        1. expiration_stdate (해당 상품의 사용기한 중 시작 날짜 YYYY-MM-DD 형식)
        2. expiration_endate (해당 상품의 사용기한 중 마감 날짜 YYYY-MM-DD 형식)
        3. room_type (객실 유형)
        4. head_count (사용 인원수, int로만 알려줘)
        5. shipping_fee (배송비, 없으면 0)
        6. transaction_method (직거래/택배거래 여부)
        7. market_price (시중 가격, int로만 알려줘)
        8. options (호텔 옵션들 항목들을 콤마로 구분)
        9. parking (주차 가능 여부, True/False)
        10. check_in_time (체크인 시간,형식 : HH:MM)
        11. check_out_time (체크아웃 시간, 형식 : HH:MM)
        12. stay_type (숙소 형태, 카테고리는 호텔, 모텔, 리조트, 풀빌라, 펜션, 카테고리 이외는 모두 기타로 분류)
        13. foreign_yn (국내 숙소인지 외국 숙소인지 알려줘, 국내/외국)
        14. event (축제/콘서트 등 관련 이벤트가 포함되어있으면 축제,콘서트 이름을 써줘)
        15. advertise (광고글 여부,숙박 시설을 판매 및 양도하는 경우는 광고글이 아니야. 대리 구매나 숙박이 아닌 다른 걸 판매하는게 광고글이야, True/False)"""
    }

    query = {
        "role": "user",
        "content": f"""
        설명에서 필요한 15개의 필드를 추출하고 JSON 형식으로 반환해줘. 단, 필드는 15개 고정이야.
        
        설명: {description}
        """
    }

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # 또는 "gpt-4" 사용 가능
            messages=[prompt, query],
            temperature=0,
            max_tokens=1000
        )

        # 응답을 받아 JSON으로 변환
        result = response['choices'][0]['message']['content']
        result = result.strip()  # 공백 제거

        # json으로 시작되는 쓸데없는 부분을 제거
        if result.startswith("```json"):
            result = result.replace("```json", "").strip()
        if result.endswith("```"):
            result = result.replace("```", "").strip()

        # Pydantic을 사용해 응답을 구조화된 데이터로 변환
        parsed_data = Fields.parse_raw(result)
        return parsed_data.dict()  # 딕셔너리로 반환

    except Exception as e:
        print("응답 파싱 중 오류 발생:", e)
        return {field: np.nan for field in Fields.__fields__}  # NaN 값으로 채운 딕셔너리 반환

In [7]:
# 멀티쓰레딩
def process_descriptions(descriptions):
    results = []
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(extract_fields, desc) for desc in descriptions]
        
        # tqdm으로 진행률 표시
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing descriptions"):
            results.append(future.result())
    
    return results

descriptions = df['description_1'].tolist()
processed_data = process_descriptions(descriptions)

Processing descriptions:  54%|█████▍    | 1039/1913 [09:13<06:49,  2.13it/s]

응답 파싱 중 오류 발생: 1 validation error for Fields
market_price
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='null', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_parsing


Processing descriptions: 100%|██████████| 1913/1913 [16:22<00:00,  1.95it/s]


# 생성 필드 합치고 저장하기

In [ ]:
processed_data

In [9]:
df1 = pd.DataFrame(processed_data)
df1

In [13]:
# df와 df1을 열 기준으로 merge
merged_df = pd.merge(df, df1, left_index=True, right_index=True)
# 필요 없는 column 삭제
merged_df = merged_df.drop(['expiration_date', 'market_price_x', 'capacity',
       'parking_x', 'options_x', 'check_in_out_time', 'shipping_fee_x',
       'transaction_location', 'transaction_method_x', 'city','city_dong'], axis=1)
# column이름 재정의
merged_df.columns = merged_df.columns.str.replace('_y', '')
merged_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 41 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   platform              1913 non-null   object        
 1   original_link         1913 non-null   object        
 2   post_time             1913 non-null   datetime64[ns]
 3   title                 1913 non-null   object        
 4   view_count            1913 non-null   int64         
 5   like_count            1913 non-null   int64         
 6   price                 1913 non-null   int64         
 7   images                1913 non-null   object        
 8   description           1913 non-null   object        
 9   category              1913 non-null   object        
 10  status                1913 non-null   object        
 11  seller_location       355 non-null    object        
 12  expiration_date       0 non-null      float64       
 13  market_price_x    

In [19]:
merged_df.to_csv('description_complete2.csv',encoding='utf-8-sig')